In [2]:
import torch
import torch.nn as nn    
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [12]:
X,y = load_iris().data, load_iris().target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.long)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset,batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=16, shuffle=False)

for i,(inputs,labels) in enumerate(train_loader):
    print(f"{inputs}------{labels}")


tensor([[ 0.6745,  0.3284,  0.8764,  1.4488],
        [ 2.2497,  1.7096,  1.6722,  1.3172],
        [-0.4160,  2.6304, -1.3402, -1.3154],
        [ 0.5533,  0.7888,  1.0469,  1.5805],
        [ 0.5533, -0.3622,  1.0469,  0.7907],
        [-0.9007,  1.7096, -1.0560, -1.0522],
        [-0.2948, -0.1320,  0.1944,  0.1325],
        [ 1.5227, -0.1320,  1.2175,  1.1856],
        [ 0.6745,  0.3284,  0.4217,  0.3958],
        [ 0.3110, -1.0528,  1.0469,  0.2641],
        [ 1.2803,  0.0982,  0.9333,  1.1856],
        [ 0.3110, -0.1320,  0.4786,  0.2641],
        [-0.7795,  0.7888, -1.3402, -1.3154],
        [ 0.5533, -1.7434,  0.3649,  0.1325],
        [ 0.9168, -0.1320,  0.3649,  0.2641],
        [ 0.5533, -0.8226,  0.6491,  0.7907]])------tensor([2, 2, 0, 2, 2, 0, 1, 2, 1, 2, 2, 1, 0, 1, 1, 2])
tensor([[-1.1430, -1.5132, -0.2603, -0.2624],
        [ 0.0687,  0.3284,  0.5922,  0.7907],
        [-0.5372,  1.9398, -1.1697, -1.0522],
        [-0.9007,  1.4794, -1.2834, -1.0522],
        [ 0.1898,

In [6]:
class IrisNet(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(4,3)
    
    def forward(self,x):
        x = self.fc1(x)
        return X 

In [13]:
model = IrisNet()
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.9)

In [14]:
epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fcn(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss+= loss.items() * inputs.size(0)
        
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}')
    

TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not numpy.ndarray